## Import Libraries

In [7]:
import pandas_datareader.data as pdr
import datetime
import numpy as np
import unittest
import pandas as pd


# seems to be a problem with pandas_datareader
# use this fix to get data from Yahoo! Finance
import fix_yahoo_finance as yf
yf.pdr_override()

# import tensorflow and print info
import tensorflow as tf
gpu_info = "Not found" if not tf.test.gpu_device_name() else tf.test.gpu_device_name() 
print("Tensorflow version %s, GPU: %s" % (tf.__version__,gpu_info))



Tensorflow version 1.2.1, GPU: Not found


## Helper Functions

In [8]:
def run_unit_tests(testClass):
    suite = unittest.TestLoader().loadTestsFromTestCase(testClass)
    unittest.TextTestRunner().run(suite)

## Data
Create a class that downloads the stock data and sets up the input/output data

In [9]:
class CreateDataSet:
    
    def __init__(self,symbol,number_of_days=10):
        self.symbol = symbol
        self.number_of_days = number_of_days
        self.train_start_date = None
        self.train_end_date = None
        self.test_start_date = None
        self.test_end_date = None
        self.validation_start_date = None
        self.validation_end_date = None
        self.yahoo_data = None
        self.pandas_df = None
        self.training_df = None
        self.test_df = None
        self.validation_df = None
        self.training_x = None
        self.training_y = None
        self.test_x = None
        self.test_y = None
        self.validation_x = None
        self.validation_y = None
        self.batch_size = None

        
    def set_training_dates(self,start_date,end_date):
        self.__check_date_string_format(start_date)
        self.__check_date_string_format(end_date)
        self.__check_start_date_before_end_date(start_date,end_date)
        self.train_start_date = start_date
        self.train_end_date = end_date
        
    def set_test_dates(self,start_date,end_date):
        self.__check_date_string_format(start_date)
        self.__check_date_string_format(end_date)
        self.__check_start_date_before_end_date(start_date,end_date)
        self.test_start_date = start_date
        self.test_end_date = end_date
        
    def set_validation_dates(self,start_date,end_date):
        self.__check_date_string_format(start_date)
        self.__check_date_string_format(end_date)
        self.__check_start_date_before_end_date(start_date,end_date)
        self.validation_start_date = start_date
        self.validation_end_date = end_date
        
    def set_batch_size(self,batch_size):
        self.batch_size = batch_size
        
    def download_data(self):
        assert self.train_start_date != None, "Training start date not set"
        assert self.train_end_date != None, "Training end date not set"
        assert self.test_start_date != None, "Test start date not set"
        assert self.test_end_date != None, "Test end date not set"
        assert self.validation_start_date != None, "Validation start date not set"
        assert self.validation_end_date != None, "Validation end date not set"
        
        start_date, end_date = self.__calculate_download_start_end_dates()

        self.yahoo_data = self.__download_data_yahoo(self.symbol,start_date,end_date)
        
        assert self.yahoo_data.shape != (0,0), "No data was downloaded"
        
    def create_dataset(self):
        
        if self.yahoo_data == None:
            self.download_data()
        
        prices = self.yahoo_data['Adj Close']
        
        # fill in missing data
        prices.fillna(method='ffill', inplace=True)
        prices.fillna(method='bfill', inplace=True)
        
        # scale features to make neural network easier to train
        scaled_prices = self.__scale_data(prices)
        
        # input features named X0, X1, X2, ...
        # output feature named Y (this is what we are predicting, tomorrows stock price)
        x_features = [ 'X%d' % (i) for i in range(self.number_of_days)]
        y_feature = ['Y']
        features = x_features + y_feature
        
        df = pd.DataFrame(index=prices.index,columns=features)
        df['Y'] = scaled_prices.shift(-1)
        
        for i in range(self.number_of_days):
            x_feature = "X%d" % (i)
            df[x_feature] = scaled_prices.shift(i)
            
        # trim the dataset to only use dates with enough data
        #
        # example: if number_of_days=10, then first date with enough
        # information will be day 10 (i.e. array index 9)
        #
        # last date will be next to last day in dataset since predicting tommorrows price
        df = df[self.number_of_days-1:-1]
        
        # there shouldn't be any NaN values
        assert df.isnull().values.any() == False, "Dataset contains unexpected NaN values"
                    
        self.pandas_df = df
        
        # split dataset into training/test/validation sets
        
        # data as pandas dataframes
        self.training_df = df[self.train_start_date:self.train_end_date]
        self.test_df = df[self.test_start_date:self.test_end_date]
        self.validation_df = df[self.validation_start_date:self.validation_end_date]
        
        # data as numpy arrays
        self.training_x = self.training_df[x_features].values
        self.training_y = self.training_df[y_feature].values
        self.test_x = self.test_df[x_features].values
        self.test_y = self.test_df[y_feature].values
        self.validation_x = self.validation_df[x_features].values
        self.validation_y = self.validation_df[y_feature].values
        
    # function used with neural networks
    # training happens on a batch instead of all the data
    # this function is called multiple times using an iterator to get the next batch
    def get_batches(self):
        assert self.batch_size != None
        
        n_batches = len(self.training_x) // self.batch_size
        assert n_batches > 0, "Batch size %d is too big (resulted in 0 batches)" % (self.batch_size)
        
        x = self.training_x[:n_batches*self.batch_size]
        y = self.training_y[:n_batches*self.batch_size]
        
        for ii in range(0, len(x), self.batch_size):
            yield x[ii:ii+self.batch_size], y[ii:ii+self.batch_size]
        
    def __download_data_yahoo(self,symbol,start_date,end_date):
        data = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
        return data
    
    def __str_to_datetime(self,date_str):
        return datetime.datetime.strptime(date_str,"%Y-%m-%d")
    
    def __check_date_string_format(self,date_str):
        assert datetime.datetime.strptime(date_str,"%Y-%m-%d"), "Dates must be a string in the format YYYY-MM-DD"
        
    def __check_start_date_before_end_date(self,start_date,end_date):
        start = self.__str_to_datetime(start_date)
        end = self.__str_to_datetime(end_date)
        assert start < end, "Start date must be before the end date"
        
    def __calculate_download_start_end_dates(self):
        train_start = self.__str_to_datetime(self.train_start_date)
        test_start = self.__str_to_datetime(self.test_start_date)
        validation_start = self.__str_to_datetime(self.validation_start_date)
        train_end = self.__str_to_datetime(self.train_end_date)
        test_end = self.__str_to_datetime(self.test_end_date)
        validation_end = self.__str_to_datetime(self.validation_end_date)
        
        start_datetime = min(train_start,test_start,validation_start)
        end_datetime = max(train_end,test_end,validation_end)
        
        start_date = start_datetime.strftime("%Y-%m-%d")
        end_date = end_datetime.strftime("%Y-%m-%d")
          
        return start_date,end_date
    
    def __scale_data(self,data):
        mean, std = data.mean(), data.std()
        self.mean = mean
        self.std = std
        
        scaled_data = (data - mean) / std
        return scaled_data
    
    def unscale_data(self,scaled_data):
        return scaled_data * self.std + self.mean
    


In [10]:
# Test the create data set class
class CreateDataSetTest(unittest.TestCase):
        
    def test_set_dates(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-01",end_date="2011-01-01")
        dataset.set_test_dates(start_date="2010-01-01",end_date="2011-01-01")
        dataset.set_validation_dates(start_date="2010-01-01",end_date="2011-01-01")
        
    def test_set_dates_not_str(self):
        dataset = CreateDataSet('SPY')
        
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date=datetime.datetime(2010,1,1),end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="2010-01-01",end_date=datetime.datetime(2011,1,1))
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date=datetime.datetime(2010,1,1),end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="2010-01-01",end_date=datetime.datetime(2011,1,1))
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date=datetime.datetime(2010,1,1),end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="2010-01-01",end_date=datetime.datetime(2011,1,1))
            
    def test_set_dates_bad_str_format(self):
        dataset = CreateDataSet('SPY')
        
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="01-01-2011",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="2010-01-01",end_date="01-01-2011")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="01-01-2011",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="2010-01-01",end_date="01-01-2011")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="01-01-2011",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="2010-01-01",end_date="01-01-2011")
            
    def test_set_dates_start_date_after_end_date(self):
        dataset = CreateDataSet('SPY')
        
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="2011-01-02",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="2011-01-02",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="2011-01-02",end_date="2011-01-01")
            
    def test_download_start_end_dates_1(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_test_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_validation_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.download_data()
        
        # (start_date,end_date) should be (training_start,validation_end)
        self.__verify_start_end_date(dataset.yahoo_data,expected_start="2010-01-04",expected_end="2012-06-01")
        
    def test_download_start_end_dates_2(self):
        dataset = CreateDataSet('SPY')
        dataset.set_test_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_validation_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_training_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.download_data()
        
        # (start_date,end_date) should be (test_start,training_end)
        self.__verify_start_end_date(dataset.yahoo_data,expected_start="2010-01-04",expected_end="2012-06-01")
        
    def test_download_start_end_dates_3(self):
        dataset = CreateDataSet('SPY')
        dataset.set_validation_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_training_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_test_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.download_data()
        
        # (start_date,end_date) should be (validation_start,test_end)
        self.__verify_start_end_date(dataset.yahoo_data,expected_start="2010-01-04",expected_end="2012-06-01")
        
    def __verify_start_end_date(self,dataframe,expected_start,expected_end):
        start_date, end_date = dataframe.index[0], dataframe.index[-1]
        start_date = start_date.strftime("%Y-%m-%d")
        end_date = end_date.strftime("%Y-%m-%d")
        
        self.assertEqual(start_date,expected_start)
        self.assertEqual(end_date,expected_end)
        
    def test_create_dataset(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_test_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_validation_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.create_dataset()
        
        # check for 10 x features and 1 y feature
        self.assertEqual(dataset.training_x.shape[1],10)
        self.assertEqual(dataset.training_y.shape[1],1)
        self.assertEqual(dataset.test_x.shape[1],10)
        self.assertEqual(dataset.test_y.shape[1],1)
        self.assertEqual(dataset.validation_x.shape[1],10)
        self.assertEqual(dataset.validation_y.shape[1],1)
        
        # check dates
        self.__check_data_in_date_range(dataset.training_df,start_date="2010-01-04",end_date="2010-06-01")
        self.__check_data_in_date_range(dataset.test_df,start_date="2011-01-04",end_date="2011-06-01")
        self.__check_data_in_date_range(dataset.validation_df,start_date="2012-01-04",end_date="2012-06-01")
        
    def __check_data_in_date_range(self,df,start_date,end_date):
        dates = df.index
        start_timestamp = pd.to_datetime(start_date)        
        end_timestamp = pd.to_datetime(end_date)
        
        for date in dates:
            self.assertTrue(date >= start_timestamp)
            self.assertTrue(date <= end_timestamp)
            
    def test_get_batches(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_test_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_validation_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.create_dataset()
        
        dataset.set_batch_size(32)
        for x_batch, y_batch in dataset.get_batches():
            self.assertEqual(x_batch.shape,(32,10))
            self.assertEqual(y_batch.shape,(32,1))
            
    
        
run_unit_tests(CreateDataSetTest)

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.....
----------------------------------------------------------------------
Ran 9 tests in 3.318s

OK


## Create your model
Setup whatever model you plan to use for price prediction. i.e. neural network, linear regression, etc.

In [20]:
class NeuralNetwork:
    
    def __init__(self):
        self.learning_rate = None
        self.number_of_inputs = None
        self.keep_prob = None
        self.epochs = None
        self.dataset = None
        self.optimizer = None
        self.cost = None
        self.hidden_layers = []
        
    def add_inputs(self,number_of_inputs):
        self.number_of_inputs = number_of_inputs
        
    def add_hidden_layer(self,number_of_nodes,dropout=False):
        self.hidden_layers.append((number_of_nodes,dropout))
            
    def set_learning_rate(self,learning_rate):
        self.learning_rate = learning_rate
        
    def set_keep_probability(self,keep_prob):
        self.keep_probability = keep_prob
        
    def set_epochs(self,epochs):
        self.epochs = epochs
        
    def set_dataset(self,dataset):
        self.dataset = dataset
        
    def build_model(self):
        
        tf.reset_default_graph()
        
        inputs = tf.placeholder(tf.float32,[None,self.number_of_inputs],name="inputs")
        targets = tf.placeholder(tf.float32,[None,1],name="targets")
        keep_prob = tf.placeholder(tf.float32,name="keep_prob")
        
        # add hidden layers
        layer = inputs
        for i in range(len(self.hidden_layers)):
            num_nodes, dropout = self.hidden_layers[i]

            layer = tf.layers.dense(layer,num_nodes,activation=tf.nn.relu)
            
            if dropout:
                layer = tf.nn.dropout(layer,keep_prob)
            
            
        # last layer predict the price
        output = tf.layers.dense(layer,1,activation=None,name="output")
        
        # setup loss function and optimizer
        loss = tf.nn.l2_loss(targets-output)
        cost = tf.reduce_mean(loss)
        optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(cost)
        
        # save variables to be used later
        self.inputs = inputs
        self.targets = targets
        self.keep_prob = keep_prob
        self.cost = cost
        self.optimizer = optimizer
        
    def train(self):
        
        self.__reset_stats()
        
        with tf.Session() as sess:
            
            sess.run(tf.global_variables_initializer())
            
            for epoch in range(self.epochs):
                
                batch_i = 0
                for batch_x, batch_y in self.dataset.get_batches():
                    
                    sess.run(self.optimizer,feed_dict={self.inputs:batch_x, self.targets:batch_y, self.keep_prob:self.keep_probability})
                    train_loss, val_loss = self.__save_stats(sess,batch_x,batch_y)
                    
                    print("Epoch %d Batch %d:  Training Loss: %0.4f, Validation Loss %0.4f" % (epoch,batch_i,train_loss,val_loss))
                    
                    batch_i += 1
            
            # run final model on the test set
            test_loss = self.__calculate_test_loss(sess)
            print("Training Complete.  Test Set Loss: %0.4f" % (test_loss))

                    
    def __reset_stats(self):
        self.train_losses = []
        self.validation_losses = []
        
    def __calculate_train_loss(self,session,batch_x,batch_y):
        train_loss = session.run(self.cost, feed_dict={self.inputs:batch_x,self.targets:batch_y,self.keep_prob:1})
        return train_loss
    
    def __calculate_validation_loss(self,session):
        validation_loss = session.run(self.cost, feed_dict={self.inputs:self.dataset.validation_x,self.targets:self.dataset.validation_y,self.keep_prob:1})
        return validation_loss
    
    def __calculate_test_loss(self,session):
        test_loss = session.run(self.cost, feed_dict={self.inputs:self.dataset.test_x,self.targets:self.dataset.test_y,self.keep_prob:1})
        return test_loss
    
                    
    def __save_stats(self,session,batch_x,batch_y):
        train_loss = self.__calculate_train_loss(session,batch_x,batch_y)
        validation_loss = self.__calculate_validation_loss(session)
        self.train_losses.append(train_loss)
        self.validation_losses.append(validation_loss)
        return train_loss, validation_loss   

In [22]:
dataset = CreateDataSet('SPY')
dataset.set_training_dates(start_date="2008-01-01",end_date="2010-01-01")
dataset.set_test_dates(start_date="2011-01-04",end_date="2011-06-01")
dataset.set_validation_dates(start_date="2012-01-04",end_date="2012-06-01") 
dataset.set_batch_size(32)
dataset.create_dataset()

nn = NeuralNetwork()
nn.set_learning_rate(.001)
nn.set_keep_probability(0.5)
nn.set_epochs(100)
nn.set_dataset(dataset)

nn.add_inputs(number_of_inputs=10)
nn.add_hidden_layer(number_of_nodes=20)
nn.build_model() 

nn.train()




[*********************100%***********************]  1 of 1 downloaded

/Users/Brent/anaconda/envs/stocks/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Epoch 0 Batch 0:  Training Loss: 52.7879, Validation Loss 175.0898
Epoch 0 Batch 1:  Training Loss: 40.3882, Validation Loss 166.3738
Epoch 0 Batch 2:  Training Loss: 69.3737, Validation Loss 157.8967
Epoch 0 Batch 3:  Training Loss: 30.5589, Validation Loss 149.8934
Epoch 0 Batch 4:  Training Loss: 14.1596, Validation Loss 142.7126
Epoch 0 Batch 5:  Training Loss: 8.4950, Validation Loss 136.5005
Epoch 0 Batch 6:  Training Loss: 14.0729, Validation Loss 131.2132
Epoch 0 Batch 7:  Training Loss: 13.2434, Validation Loss 126.6611
Epoch 0 Batch 8:  Training Loss: 21.7701, Validation Loss 122.7163
Epoch 0 Batch 9:  Training Loss: 14.2971, Validation Loss 119.2700
Epoch 0 Batch 10:  Training Loss: 6.9695, Validation Loss 116.2368
Epoch 0 Batch 11:  Training Loss: 5.9472, Validation Loss 113.5542
Epoch 0 Batch 12:  Training Loss: 1.9646, Validation Loss 111.1694
Epoch 0 Batch 13:  Training Loss: 0.6522, Validation Loss 109.0402
Epoch 0 Batch 14:  Training Loss: 0.5113, Validation Loss 107.1

Epoch 14 Batch 0:  Training Loss: 0.2852, Validation Loss 0.5792
Epoch 14 Batch 1:  Training Loss: 0.3809, Validation Loss 0.5786
Epoch 14 Batch 2:  Training Loss: 0.2850, Validation Loss 0.5687
Epoch 14 Batch 3:  Training Loss: 0.3395, Validation Loss 0.5655
Epoch 14 Batch 4:  Training Loss: 0.1847, Validation Loss 0.5632
Epoch 14 Batch 5:  Training Loss: 3.1134, Validation Loss 0.5646
Epoch 14 Batch 6:  Training Loss: 1.5033, Validation Loss 0.5663
Epoch 14 Batch 7:  Training Loss: 0.5272, Validation Loss 0.5678
Epoch 14 Batch 8:  Training Loss: 0.6220, Validation Loss 0.5692
Epoch 14 Batch 9:  Training Loss: 0.3842, Validation Loss 0.5709
Epoch 14 Batch 10:  Training Loss: 0.2514, Validation Loss 0.5727
Epoch 14 Batch 11:  Training Loss: 0.3259, Validation Loss 0.5743
Epoch 14 Batch 12:  Training Loss: 0.1692, Validation Loss 0.5759
Epoch 14 Batch 13:  Training Loss: 0.2117, Validation Loss 0.5773
Epoch 14 Batch 14:  Training Loss: 0.2081, Validation Loss 0.5787
Epoch 15 Batch 0:  T

Epoch 27 Batch 0:  Training Loss: 0.2766, Validation Loss 0.5275
Epoch 27 Batch 1:  Training Loss: 0.3773, Validation Loss 0.5264
Epoch 27 Batch 2:  Training Loss: 0.2428, Validation Loss 0.5153
Epoch 27 Batch 3:  Training Loss: 0.3108, Validation Loss 0.5124
Epoch 27 Batch 4:  Training Loss: 0.1780, Validation Loss 0.5099
Epoch 27 Batch 5:  Training Loss: 2.4491, Validation Loss 0.5117
Epoch 27 Batch 6:  Training Loss: 1.3362, Validation Loss 0.5136
Epoch 27 Batch 7:  Training Loss: 0.4579, Validation Loss 0.5153
Epoch 27 Batch 8:  Training Loss: 0.5299, Validation Loss 0.5169
Epoch 27 Batch 9:  Training Loss: 0.3120, Validation Loss 0.5184
Epoch 27 Batch 10:  Training Loss: 0.2203, Validation Loss 0.5196
Epoch 27 Batch 11:  Training Loss: 0.2872, Validation Loss 0.5208
Epoch 27 Batch 12:  Training Loss: 0.1617, Validation Loss 0.5218
Epoch 27 Batch 13:  Training Loss: 0.2034, Validation Loss 0.5226
Epoch 27 Batch 14:  Training Loss: 0.1946, Validation Loss 0.5232
Epoch 28 Batch 0:  T

Epoch 39 Batch 4:  Training Loss: 0.1771, Validation Loss 0.4613
Epoch 39 Batch 5:  Training Loss: 1.9609, Validation Loss 0.4629
Epoch 39 Batch 6:  Training Loss: 1.2475, Validation Loss 0.4647
Epoch 39 Batch 7:  Training Loss: 0.4156, Validation Loss 0.4664
Epoch 39 Batch 8:  Training Loss: 0.4797, Validation Loss 0.4679
Epoch 39 Batch 9:  Training Loss: 0.2683, Validation Loss 0.4693
Epoch 39 Batch 10:  Training Loss: 0.1985, Validation Loss 0.4705
Epoch 39 Batch 11:  Training Loss: 0.2544, Validation Loss 0.4715
Epoch 39 Batch 12:  Training Loss: 0.1546, Validation Loss 0.4723
Epoch 39 Batch 13:  Training Loss: 0.1914, Validation Loss 0.4730
Epoch 39 Batch 14:  Training Loss: 0.1883, Validation Loss 0.4734
Epoch 40 Batch 0:  Training Loss: 0.2668, Validation Loss 0.4741
Epoch 40 Batch 1:  Training Loss: 0.3666, Validation Loss 0.4727
Epoch 40 Batch 2:  Training Loss: 0.2074, Validation Loss 0.4620
Epoch 40 Batch 3:  Training Loss: 0.2716, Validation Loss 0.4597
Epoch 40 Batch 4:  T

Epoch 53 Batch 5:  Training Loss: 1.5552, Validation Loss 0.4210
Epoch 53 Batch 6:  Training Loss: 1.1734, Validation Loss 0.4226
Epoch 53 Batch 7:  Training Loss: 0.3747, Validation Loss 0.4241
Epoch 53 Batch 8:  Training Loss: 0.3965, Validation Loss 0.4254
Epoch 53 Batch 9:  Training Loss: 0.2385, Validation Loss 0.4266
Epoch 53 Batch 10:  Training Loss: 0.1736, Validation Loss 0.4277
Epoch 53 Batch 11:  Training Loss: 0.2245, Validation Loss 0.4286
Epoch 53 Batch 12:  Training Loss: 0.1418, Validation Loss 0.4293
Epoch 53 Batch 13:  Training Loss: 0.1693, Validation Loss 0.4299
Epoch 53 Batch 14:  Training Loss: 0.1769, Validation Loss 0.4304
Epoch 54 Batch 0:  Training Loss: 0.2605, Validation Loss 0.4312
Epoch 54 Batch 1:  Training Loss: 0.3579, Validation Loss 0.4298
Epoch 54 Batch 2:  Training Loss: 0.1787, Validation Loss 0.4204
Epoch 54 Batch 3:  Training Loss: 0.2350, Validation Loss 0.4189
Epoch 54 Batch 4:  Training Loss: 0.1786, Validation Loss 0.4167
Epoch 54 Batch 5:  T

Epoch 65 Batch 6:  Training Loss: 1.1203, Validation Loss 0.3761
Epoch 65 Batch 7:  Training Loss: 0.3516, Validation Loss 0.3769
Epoch 65 Batch 8:  Training Loss: 0.3524, Validation Loss 0.3772
Epoch 65 Batch 9:  Training Loss: 0.2277, Validation Loss 0.3776
Epoch 65 Batch 10:  Training Loss: 0.1602, Validation Loss 0.3781
Epoch 65 Batch 11:  Training Loss: 0.2054, Validation Loss 0.3786
Epoch 65 Batch 12:  Training Loss: 0.1328, Validation Loss 0.3790
Epoch 65 Batch 13:  Training Loss: 0.1519, Validation Loss 0.3794
Epoch 65 Batch 14:  Training Loss: 0.1650, Validation Loss 0.3797
Epoch 66 Batch 0:  Training Loss: 0.2553, Validation Loss 0.3804
Epoch 66 Batch 1:  Training Loss: 0.3383, Validation Loss 0.3792
Epoch 66 Batch 2:  Training Loss: 0.1546, Validation Loss 0.3719
Epoch 66 Batch 3:  Training Loss: 0.1996, Validation Loss 0.3710
Epoch 66 Batch 4:  Training Loss: 0.1845, Validation Loss 0.3693
Epoch 66 Batch 5:  Training Loss: 1.3119, Validation Loss 0.3703
Epoch 66 Batch 6:  T

Epoch 79 Batch 8:  Training Loss: 0.6288, Validation Loss 0.3214
Epoch 79 Batch 9:  Training Loss: 0.2023, Validation Loss 0.3203
Epoch 79 Batch 10:  Training Loss: 0.1368, Validation Loss 0.3193
Epoch 79 Batch 11:  Training Loss: 0.1959, Validation Loss 0.3186
Epoch 79 Batch 12:  Training Loss: 0.1552, Validation Loss 0.3180
Epoch 79 Batch 13:  Training Loss: 0.1813, Validation Loss 0.3174
Epoch 79 Batch 14:  Training Loss: 0.1943, Validation Loss 0.3169
Epoch 80 Batch 0:  Training Loss: 0.2558, Validation Loss 0.3171
Epoch 80 Batch 1:  Training Loss: 0.3210, Validation Loss 0.3166
Epoch 80 Batch 2:  Training Loss: 0.1271, Validation Loss 0.3145
Epoch 80 Batch 3:  Training Loss: 0.1830, Validation Loss 0.3149
Epoch 80 Batch 4:  Training Loss: 0.1901, Validation Loss 0.3150
Epoch 80 Batch 5:  Training Loss: 1.1674, Validation Loss 0.3162
Epoch 80 Batch 6:  Training Loss: 1.2288, Validation Loss 0.3175
Epoch 80 Batch 7:  Training Loss: 0.4786, Validation Loss 0.3193
Epoch 80 Batch 8:  T

Epoch 91 Batch 4:  Training Loss: 0.1870, Validation Loss 0.2719
Epoch 91 Batch 5:  Training Loss: 1.1001, Validation Loss 0.2725
Epoch 91 Batch 6:  Training Loss: 1.0778, Validation Loss 0.2729
Epoch 91 Batch 7:  Training Loss: 0.3546, Validation Loss 0.2733
Epoch 91 Batch 8:  Training Loss: 0.4154, Validation Loss 0.2735
Epoch 91 Batch 9:  Training Loss: 0.2405, Validation Loss 0.2738
Epoch 91 Batch 10:  Training Loss: 0.1670, Validation Loss 0.2742
Epoch 91 Batch 11:  Training Loss: 0.2054, Validation Loss 0.2746
Epoch 91 Batch 12:  Training Loss: 0.1153, Validation Loss 0.2751
Epoch 91 Batch 13:  Training Loss: 0.1390, Validation Loss 0.2756
Epoch 91 Batch 14:  Training Loss: 0.1450, Validation Loss 0.2760
Epoch 92 Batch 0:  Training Loss: 0.2396, Validation Loss 0.2768
Epoch 92 Batch 1:  Training Loss: 0.3439, Validation Loss 0.2762
Epoch 92 Batch 2:  Training Loss: 0.1372, Validation Loss 0.2739
Epoch 92 Batch 3:  Training Loss: 0.1529, Validation Loss 0.2739
Epoch 92 Batch 4:  T

## Train your model
Define the function to train your model using the training data and the test data

In [ ]:
def train(model,train_data,test_data):
    pass

## Evaluate your model on the validation set
Return the accuracy of your model

In [ ]:
def evaluate(model,validation_data):
    pass

## Display the results
Show graphs and metrics of your model accuracy

In [ ]:
def results():
    pass

## Train and evaluate your model here
Call the functions to get the data, train the model, and evaluate it on the validation data